In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS

ModuleNotFoundError: No module named 'rasterio'

Select path to .img file. The code will then read the .xml and write the .tif to the same location

In [ ]:
image_path = input("Enter the path to the image file: ")

In [ ]:
xml_file_path = image_path[:-3] + 'xml'

namespaces = {
    'pds': 'http://pds.nasa.gov/pds4/pds/v1',
    'isda': 'https://isda.issdc.gov.in/pds4/isda/v1'
}

tree = ET.parse(xml_file_path)
root = tree.getroot()

line_element = root.find(".//pds:Axis_Array[pds:axis_name='Line']/pds:elements", namespaces=namespaces)
sample_element = root.find(".//pds:Axis_Array[pds:axis_name='Sample']/pds:elements", namespaces=namespaces)

if line_element is not None:
    line_value = int(line_element.text)
    print(f"Line Value: {line_value}")
else:
    print("Line element not found in XML.")

if sample_element is not None:
    sample_value = int(sample_element.text)
    print(f"Sample Value: {sample_value}")
else:
    print("Sample element not found in XML.")

upper_left_lat = float(root.find(".//isda:upper_left_latitude", namespaces=namespaces).text)
upper_left_lon = float(root.find(".//isda:upper_left_longitude", namespaces=namespaces).text)

lower_left_lat = float(root.find(".//isda:lower_left_latitude", namespaces=namespaces).text)
lower_left_lon = float(root.find(".//isda:lower_left_longitude", namespaces=namespaces).text)

pixel_width = (upper_left_lon - lower_left_lon) / sample_value
pixel_height = (upper_left_lat - lower_left_lat) / line_value
pixel_size = (pixel_width + pixel_height) / 2.0

image_width = sample_value
image_height = line_value

data_type = np.uint8

with open(image_path, 'rb') as file:
    raw_data = np.fromfile(file, dtype=data_type, count=image_width * image_height)

image_array = raw_data.reshape((image_height, image_width))

output_geotiff = image_path[:-3]+'tif'

transform = from_origin(lower_left_lon, upper_left_lat, pixel_size, -pixel_size)  # Use -pixel_size for latitude

with rasterio.open(
        output_geotiff,
        'w',
        driver='GTiff',
        height=image_height,
        width=image_width,
        count=1,
        dtype=data_type,
        crs=CRS.from_wkt('GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]]'),
        transform=transform
) as dst:
    dst.write(image_array, 1)

print(f"GeoTIFF file saved as: {output_geotiff}")

plt.imshow(image_array, cmap='gray')
plt.axis('off')
plt.show()